In [ ]:
import pandas as pd

temp = pd.read_excel("/content/시군구별_가뭄도데이터.csv")


In [ ]:
temp = temp.groupby(["Unnamed: 1", "Unnamed: 3"], as_index=False)[["SIGUNGU_CD"]].mean()

In [ ]:
temp["SIGUNGU_CD"] = temp["2024년6월 행정구역 코드"].astype(str) + temp["Unnamed: 2"].astype(str)

In [ ]:
temp["SIGUNGU_CD"] = temp["SIGUNGU_CD"].astype(int)

In [ ]:
temp

In [ ]:
temp.columns = ["시도", "시군구", "SIGUNGU_CD"]

In [ ]:
temp.to_csv("시군구_코드.csv")

#Total

In [ ]:
import pandas as pd

df = pd.read_csv("/content/시군구별_가뭄도데이터.csv")

In [ ]:
# 'Unnamed'로 시작하는 모든 열 제거
df = df.loc[:, ~df.columns.str.startswith('Unnamed')]


In [ ]:
df

In [ ]:
feature_cols = [ '대응능력 계수(단위: 점수)', '노출도 계수(단위: 무차원)', '민감도 계수(단위: 무차원)',
       '보조수원 계수(단위: 무차원)', '취약성(단위: 점수)']

# nnPU

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1. 데이터 준비
X = df[feature_cols].values  # 사용할 feature 컬럼
y = df["사업여부"].fillna(0).values  # 1: Positive, 0: Unlabeled

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train)
X_val = torch.FloatTensor(X_val)
y_val = torch.FloatTensor(y_val)

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.linear = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.linear(x)

def nnPU_loss(y_pred, y_true, pi=0.1, beta=0):
    positive = y_true == 1
    unlabeled = y_true == 0

    # Positive risk
    pos_pred = y_pred[positive]
    pos_loss = torch.mean(-torch.log(pos_pred + 1e-8))

    # Unlabeled risk
    unl_pred = y_pred[unlabeled]
    neg_loss = torch.mean(-torch.log(1 - unl_pred + 1e-8))

    # PU risk with non-negative constraint
    pu_loss = pi * pos_loss - pi * neg_loss
    return torch.maximum(pu_loss, torch.tensor(beta))

In [ ]:
model = SimpleNN(X_train.shape[1])
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    y_pred = model(X_train).squeeze()
    loss = nnPU_loss(y_pred, y_train, pi=y_train.mean().item())
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

In [ ]:
# df와 동일한 인덱스를 가진 df_val을 따로 구성했어야 함
X_train, X_val, y_train, y_val, df_train, df_val = train_test_split(
    X_scaled, y, df, test_size=0.2, stratify=y, random_state=42
)

# 예측
model.eval()
with torch.no_grad():
    probs = model(torch.FloatTensor(X_val)).squeeze().numpy()

# 결과 저장
df_val = df_val.copy()  # 복사해서 안전하게
df_val["사업확률"] = probs

# 상위 20개 보기
top_candidates = df_val.sort_values("사업확률", ascending=False).head(20)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns


from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt

# 예측 라벨 (threshold 0.5 기준)
y_pred = (probs > 0.5).astype(int)

# 혼동행렬
cm = confusion_matrix(y_val, y_pred)
plt.figure(figsize=(4, 3))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

# 분류 리포트
print(classification_report(y_val, y_pred))

# ROC-AUC
roc_auc = roc_auc_score(y_val, probs)
print(f"ROC-AUC Score: {roc_auc:.4f}")

In [ ]:
sns.histplot(probs, bins=20, kde=True)
plt.title("사업 확률 분포")  # 한글도 깨지지 않음
plt.xlabel("사업 확률")
plt.ylabel("건수")
plt.show()

# Logistic

In [ ]:
# 📌 1. 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler


df = merged.copy()

# 📌 3. Feature, Target 설정
feature_cols = [
    "대응능력 계수(단위: 점수)",
    "노출도 계수(단위: 무차원)",
    "민감도 계수(단위: 무차원)",
    "보조수원 계수(단위: 무차원)",
    "취약성(단위: 점수)"
]
X = df[feature_cols]
y = df["사업여부"].fillna(0).astype(int)

# 📌 4. 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 📌 5. 훈련/검증 분리
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, stratify=y, test_size=0.2, random_state=42)

# 📌 6. 로지스틱 회귀 모델 학습
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train, y_train)

# 📌 7. 예측 및 평가
y_pred = lr.predict(X_val)
y_prob = lr.predict_proba(X_val)[:, 1]

print("📌 ROC AUC:", roc_auc_score(y_val, y_prob))
print("📌 Classification Report:\n", classification_report(y_val, y_pred))

# 📌 8. 혼동행렬 시각화
plt.figure(figsize=(4, 3))
sns.heatmap(confusion_matrix(y_val, y_pred), annot=True, fmt='d', cmap='Greens')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix (Logistic Regression)")
plt.tight_layout()
plt.show()

# 📌 9. 전체 데이터 사업 확률 예측
df["사업확률"] = lr.predict_proba(X_scaled)[:, 1]

# 📌 10. 사업이 아직 안 된 곳 중 확률 높은 상위 20개 추출
top20 = df[df["사업여부"] == 0].sort_values("사업확률", ascending=False)

# 📌 11. 시각화
plt.figure(figsize=(8, 6))
sns.barplot(data=top20, y="시군구", x="사업확률", palette="crest")
plt.title("사업 유력 후보 상위 20개 지역 (로지스틱)")
plt.xlabel("사업 확률")
plt.ylabel("시군구")
plt.tight_layout()
plt.show()

# 📌 12. 상위 20개 출력
top20

In [ ]:
import geopandas as gpd

top20 = gpd.GeoDataFrame(top20, geometry=top20["geometry"])

In [ ]:
top20

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'AppleGothic'  # 맥 기본 한글 폰트
plt.rcParams['axes.unicode_minus'] = False



# 지도 시각화
fig, ax = plt.subplots(figsize=(18, 24), facecolor='white',edgecolor='none')  # 전체 배경 검정

# 배경 전체 회색 또는 흰색으로 칠하지 않음
gdf.plot(ax=ax, color='whitesmoke', edgecolor='none', linewidth=0.3)

# anomaly_score가 있는 시군구만 색칠
top20.plot(
    column='사업확률',
    ax=ax,
    edgecolor='none',   # 경계선 없음
    cmap='YlOrBr',
    linewidth=0,
    legend=False,
)

# 스타일 설정
plt.title("시군구별 예측 점수", fontsize=26, color='Black')
plt.axis('off')
plt.show()


In [ ]:
sorting = top20[["시도", "시군구", "코드", "사업확률"]]

In [ ]:
sorting = sorting.reset_index(drop=True)

In [ ]:
sorting

In [ ]:
sorting.to_csv("ranking.csv")

#RandomForest

In [ ]:
# 📌 1. 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler

# 📌 2. 데이터 불러오기
df = pd.read_csv("/content/최종가공데이터.csv")

# 📌 3. Feature, Target 설정
feature_cols = [
    "대응능력 계수(단위: 점수)",
    "노출도 계수(단위: 무차원)",
    "민감도 계수(단위: 무차원)",
    "보조수원 계수(단위: 무차원)",
    "취약성(단위: 점수)"
]
X = df[feature_cols]
y = df["사업여부"].fillna(0).astype(int)  # NaN은 0 처리

# 📌 4. 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 📌 5. 훈련/검증 분리
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, stratify=y, test_size=0.2, random_state=42)

# 📌 6. 모델 학습
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# 📌 7. 예측 및 평가
y_pred = rf.predict(X_val)
y_prob = rf.predict_proba(X_val)[:, 1]

print("📌 ROC AUC:", roc_auc_score(y_val, y_prob))
print("📌 Classification Report:\n", classification_report(y_val, y_pred))

# 📌 8. 혼동행렬 시각화
plt.figure(figsize=(4, 3))
sns.heatmap(confusion_matrix(y_val, y_pred), annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()

# 📌 9. 전체 데이터 사업 확률 예측
df["사업확률"] = rf.predict_proba(X_scaled)[:, 1]

# 📌 10. 사업이 아직 안 된 곳 중에서 확률 높은 상위 20개 추출
top20 = df[df["사업여부"] == 0].sort_values("사업확률", ascending=False).head(20)

# 📌 11. 시각화
plt.figure(figsize=(8, 6))
sns.barplot(data=top20, y="시군구", x="사업확률", palette="viridis")
plt.title("사업 유력 후보 상위 20개 지역")
plt.xlabel("사업 확률")
plt.ylabel("시군구")
plt.tight_layout()
plt.show()

# 📌 12. 상위 20개 출력
top20[["시도", "시군구", "사업확률"]]

# Geo 시각화


In [ ]:
import geopandas as gpd

df = pd.read_csv("/content/시군구별_가뭄도데이터.csv", index_col=0)
shp_path = "/content/BND_SIGUNGU_PG.shp"
gdf = gpd.read_file(shp_path)

In [ ]:
gdf[gdf["SIGUNGU_NM"] == "중구"]

In [ ]:
df = df[['시도', '시군구', '코드', '대응능력 계수(단위: 점수)', '노출도 계수(단위: 무차원)',
       '민감도 계수(단위: 무차원)', '보조수원 계수(단위: 무차원)', '취약성(단위: 점수)', '사업여부']]

In [ ]:
df["코드"] = df["코드"].astype(str)

In [ ]:
merged = df.merge(gdf, how='left', left_on='코드', right_on='SIGUNGU_CD')


merged

In [ ]:
merged.to_csv("temp.csv")

# 데이터 가공


In [ ]:
for i in data:
    if i[0].isnumeric():
        i.pop(0)

print(data)

In [ ]:
for i in data:
    if len(i) > 7:
        print(i)

In [ ]:
import pandas as pd

# 컬럼명 지정
columns = [
    "행정동명",
    "대응능력 계수(단위: 점수)", "노출도 계수(단위: 무차원)", "민감도 계수(단위: 무차원)",
    "보조수원 계수(단위: 무차원)", "취약성(단위: 점수)", "취약성 등급(단위: 등급)"
]

# DataFrame으로 변환
df = pd.DataFrame(data, columns=columns)

In [ ]:
df.head(100)

In [ ]:
df["취약성 등급(단위: 등급)"].value_counts(dropna=False)

In [ ]:
print(df.loc[df["취약성 등급(단위: 등급)"] == None])

In [ ]:
import pandas as pd
import re

def split_address(addr):
    # 세종시
    if addr.startswith("세종특별자치시"):
        return pd.Series(["세종특별자치시", "", addr[len("세종특별자치시"):]])

    # 시도 추출
    match_sido = re.match(r"^(.*?[도시])", addr)
    시도 = match_sido.group(1) if match_sido else ""
    나머지 = addr[len(시도):]

    # 전체가 행정동 하나로 끝날 수 있는 경우: '동', '면', '읍' 으로 끝나면
    match_dong = re.match(r"^(.*[시군구])(.+[동읍면])$", 나머지)
    if match_dong:
        시군구 = match_dong.group(1)
        행정동 = match_dong.group(2)
    else:
        # 일반적인 경우 fallback
        match_sigungu = re.match(r"^(.*(시|군|구))", 나머지)
        시군구 = match_sigungu.group(1) if match_sigungu else ""
        행정동 = 나머지[len(시군구):]

    return pd.Series([시도, 시군구, 행정동])

In [ ]:
df[["시도", "시군구", "행정동"]] = df["행정동명"].apply(split_address)

# 결과 확인
print(df)

In [ ]:
df.head(20)

In [ ]:
df = df[["시도", "시군구", "행정동",
         "대응능력 계수(단위: 점수)", "노출도 계수(단위: 무차원)", "민감도 계수(단위: 무차원)",
         "보조수원 계수(단위: 무차원)", "취약성(단위: 점수)", "취약성 등급(단위: 등급)"]]

In [ ]:
df.head(500)

In [ ]:
df.to_csv("데이터.csv")

In [ ]:
cols_to_float = [
    "대응능력 계수(단위: 점수)",
    "노출도 계수(단위: 무차원)",
    "민감도 계수(단위: 무차원)",
    "보조수원 계수(단위: 무차원)",
    "취약성(단위: 점수)"
]
for col in cols_to_float:
    df.loc[:, col] = pd.to_numeric(df[col], errors="coerce")

In [ ]:
df

In [ ]:
# 조건: 시군구가 '시구'로 끝나는 경우
mask = df['시군구'].str.endswith('강남구압구')

# 시군구에서 '구'를 떼고, 행정동 앞에 붙이기
df.loc[mask, '행정동'] = '압구' + df.loc[mask, '행정동']
df.loc[mask, '시군구'] = df.loc[mask, '시군구'].str[:-2]  # '시구' -> '시'

In [ ]:
df.loc[df["시군구"] == "강남구"]

In [ ]:
# 조건: 시군구가 '시구'로 끝나는 경우
mask = df['시군구'].str.endswith('남동구구')

# 시군구에서 '구'를 떼고, 행정동 앞에 붙이기
df.loc[mask, '행정동'] = '구' + df.loc[mask, '행정동']
df.loc[mask, '시군구'] = df.loc[mask, '시군구'].str[:-1]  # '시구' -> '시'

In [ ]:
df.loc[df["시군구"] == "남동구"]

In [ ]:
df = df.groupby(["시도", "시군구"])[["대응능력 계수(단위: 점수)", "노출도 계수(단위: 무차원)", "민감도 계수(단위: 무차원)","보조수원 계수(단위: 무차원)", "취약성(단위: 점수)"]].mean().reset_index()

In [ ]:
df

In [ ]:
df.to_csv("제주도.csv")

In [ ]:
seoul = df.iloc[:425,:]
incheon = df.iloc[425:580,:]
gyeonggi = df.iloc[580:,:]

In [ ]:
seoul = seoul.groupby(["시도", "시군구"])[["대응능력 계수(단위: 점수)", "노출도 계수(단위: 무차원)", "민감도 계수(단위: 무차원)","보조수원 계수(단위: 무차원)", "취약성(단위: 점수)"]].mean().reset_index()

In [ ]:
seoul

In [ ]:
incheon = incheon.groupby(["시도", "시군구"])[["대응능력 계수(단위: 점수)", "노출도 계수(단위: 무차원)", "민감도 계수(단위: 무차원)","보조수원 계수(단위: 무차원)", "취약성(단위: 점수)"]].mean().reset_index()

In [ ]:
incheon

In [ ]:
gyeongi = gyeonggi.groupby(["시도", "시군구"])[["대응능력 계수(단위: 점수)", "노출도 계수(단위: 무차원)", "민감도 계수(단위: 무차원)","보조수원 계수(단위: 무차원)", "취약성(단위: 점수)"]].mean().reset_index()

In [ ]:
# 조건: 시군구가 '시구'로 끝나는 경우
mask = df['시군구'].str.endswith('포천시군')

# 시군구에서 '구'를 떼고, 행정동 앞에 붙이기
df.loc[mask, '행정동'] = '군' + df.loc[mask, '행정동']
df.loc[mask, '시군구'] = df.loc[mask, '시군구'].str[:-1]  # '시구' -> '시'

In [ ]:
gyeongi

In [ ]:
gyeongbuk = gyeongbuk.groupby(["시도", "시군구"])[["대응능력 계수(단위: 점수)", "노출도 계수(단위: 무차원)", "민감도 계수(단위: 무차원)","보조수원 계수(단위: 무차원)", "취약성(단위: 점수)"]].mean().reset_index()

In [ ]:
gyeongbuk

In [ ]:
gyeongnam = gyeongnam.groupby(["시도", "시군구"])[["대응능력 계수(단위: 점수)", "노출도 계수(단위: 무차원)", "민감도 계수(단위: 무차원)","보조수원 계수(단위: 무차원)", "취약성(단위: 점수)"]].mean().reset_index()

In [ ]:
gyeongnam

In [ ]:
df_all = pd.concat([seoul, incheon, gyeongi], axis=0, ignore_index=True)

In [ ]:
df_all

In [ ]:
df_all = pd.concat([seoul, incheon, gyeongi], axis=0, ignore_index=True)

In [ ]:
df_all.to_csv("수도권.csv")

In [ ]:
sudo = pd.read_csv("/content/수도권.csv")
gangwon = pd.read_csv("/content/강원도.csv")
gyeongsang = pd.read_csv("/content/경상권.csv")
chungcheong = pd.read_csv("/content/충청권_가뭄도.csv")
junla = pd.read_csv("/content/전라권.csv")
jeju = pd.read_csv("/content/제주도.csv")

In [ ]:
df_all = pd.concat([sudo, gangwon, gyeongsang, chungcheong, junla, jeju], axis=0, ignore_index=True)

In [ ]:
df_all.head(10)

In [ ]:
columns = ["시도", "시군구","대응능력 계수(단위: 점수)",	"노출도 계수(단위: 무차원)",
           "민감도 계수(단위: 무차원)",	"보조수원 계수(단위: 무차원)",	"취약성(단위: 점수)"]

In [ ]:
df = df_all[columns]

In [ ]:
3 > 2

In [ ]:
df.head(10)

In [ ]:
df.to_csv("total.csv")

In [ ]:
# 조건: 시군구가 '시구'로 끝나는 경우
mask = junla['시군구'].str.endswith('진도군군')

# 시군구에서 '구'를 떼고, 행정동 앞에 붙이기
junla.loc[mask, '시군구'] = junla.loc[mask, '시군구'].str[:-1]  # '시구' -> '시'

In [ ]:
junla.loc[junla["시도"] == "전라남도"]

In [ ]:
junla = junla.groupby(["시도", "시군구"])[["대응능력 계수(단위: 점수)", "노출도 계수(단위: 무차원)", "민감도 계수(단위: 무차원)","보조수원 계수(단위: 무차원)", "취약성(단위: 점수)"]].mean().reset_index()

In [ ]:
junla

In [ ]:
df_all.to_csv("total.csv")

# 취합

In [ ]:
import pandas as pd

# 가뭄 마이닝

In [ ]:
gamum = pd.read_csv("/content/가뭄도_랭킹.csv").iloc[:,1:]
gamum.head(10)

In [ ]:
gamum["가뭄등수"] = range(1, len(gamum) + 1)

gamum

In [ ]:
gamum = gamum[["시도", "시군구", "가뭄등수"]]
gamum

# 수리 마이닝

In [ ]:
sangsu = pd.read_csv("/content/수리상수_랭킹.csv")
sangsu.head(10)

In [ ]:
sangsu = sangsu[sangsu["SIGUNGU_NM"].notna()].reset_index(drop=True)
sangsu.columns = ["상수등수", "시도", "시군구", "확률"]
sangsu["상수등수"] = range(1, len(sangsu) + 1)

sangsu

In [ ]:
sangsu = sangsu[["시도", "시군구", "상수등수"]]

sangsu = sangsu[:148]
sangsu

# 두 개 먼저 merge

In [ ]:
two = sangsu.merge(gamum, on=["시도", "시군구"], how="left")
two

In [ ]:
two.to_csv("temp.csv")

# 수질

In [ ]:
sugil = pd.read_csv("/content/수질_랭킹.csv")

In [ ]:
sugil

In [ ]:
sugil["수질등수"] = range(1, len(sugil) + 1)

sugil

In [ ]:
sugil = sugil[["시도", "시군구", "수질등수"]]
sugil

In [ ]:
sugil[sugil["시도"] == "제주도"]

In [ ]:
three = two.merge(sugil, on=["시도", "시군구"], how="left")
three

In [ ]:
three.to_csv("랭킹.csv")

In [ ]:
ranking = pd.read_csv("//content/랭킹.csv", index_col=0)

In [ ]:
ranking

In [ ]:
ranking = ranking[ranking["시군구"].notna()].reset_index(drop=True)

In [ ]:
ranking

In [ ]:
ranking["최종등수"] = (ranking["상수등수"] + ranking["가뭄등수"] + ranking["수질등수"])/3
ranking

In [ ]:
ranking = ranking.sort_values(by="최종등수", ascending=True)
ranking

In [ ]:
ranking["최종등수"] = range(1, len(ranking) + 1)
ranking

In [ ]:
ranking.to_csv("최종랭킹.csv")